# 第7章 图像复原
- 图像采集过程中，可能因复杂成像因素而降质，例如：传感器噪声、照相机镜头失焦、照相机与目标之间的相对运动、大气湍流的随机扰动、雾霾等原因导致图像模糊，这种现象称为图像退化（Image degradation）。
- 图像退化将影响图像后继处理过程，增加图像计算、分析、特征提取及目标识别的难度，降低图像数据的应用价值。
- 根据图像退化过程的先验知识，建立退化过程的数学模型，对退化图像进行修复或者重建，称为图像复原（Image restoration）

In [1]:
#导入本章示例所用到的包,使用本文档中示例,先运行一次本段代码
import numpy as np
from scipy.fft import fft,ifft,fft2,ifft2,fftshift,ifftshift
import cv2 as cv
from skimage import io,util,filters,restoration
import matplotlib.pyplot as plt
%matplotlib inline

## 7.1 图像退化过程及其模型化方法
### 1.大气湍流图像模糊退化模型 Simulation of atmospheric turbulence
- #### 先运行函数定义cell装载函数，才能调用

In [2]:
def AtmoTurbulenceSim(image,k):
    """
    模拟大气湍流图像模糊退化AtmoTurbulence
    输入参数：
            image - 原图像, 灰度图像或RGB彩色图像;
                k - 大气湍流模型的系数,k越大意味大气湍流越剧烈,导致图像越模糊;
    输出： 
           imgout - 大气湍流影响的退化图像;
             Hatm - 大气湍流模糊退化函数;
    """

    #获取图像高/宽
    rows,cols = image.shape[0:2]
    
    #采用'reflect'方式扩展图像(下面扩展rows行,右面扩张cols列)
    if image.ndim==3:
        #彩色图像
        imgex = np.pad(image,((0,rows),(0,cols),(0,0)),mode='reflect')
    elif image.ndim==2:
        #灰度图像        
        imgex = np.pad(image,((0,rows),(0,cols)),mode='reflect')
    
    #计算扩展图像的DFT,并中心化
    img_dft = fft2(imgex,axes=(0,1))
    img_dft = fftshift(img_dft,axes=(0,1))
    
    #生成大气湍流模糊退化函数
    #构建频域平面坐标网格数组，坐标轴定义v列向/u行向
    v = np.arange(-cols, cols) 
    u = np.arange(-rows, rows)
    Va, Ua = np.meshgrid(v, u)

    D2 = Ua ** 2 + Va ** 2
    Hatm = np.exp(-k *(D2 ** (5.0/6.0)))
    
    if image.ndim==3:
        #彩色图像把H串接成三维数组
        Hatm = np.dstack(( Hatm, Hatm, Hatm)) 
        
    #计算图像DFT与大气湍流模糊退化函数的点积
    Gp= img_dft * Hatm
    #去中心化
    Gp = ifftshift(Gp,axes=(0,1))    
    #DFT反变换,并取实部
    imgp = np.real(ifft2(Gp,axes=(0,1)))
    #把输出图像的数据格式转换为uint8
    imgp = np.uint8(np.clip(imgp,0,255))
    #截取imgp左上角与原图像大小相等的区域作为输出
    imgout = imgp[0:rows,0:cols]
    
    return imgout,Hatm    
#----------------------------------------------------

In [ ]:
#采用自定义函数AtmoTurbulenceSim模拟大气湍流模糊退化

#读入一幅航拍图片
img = io.imread('./imagedata/aerial_image.png')

#令k=0.001,模拟大气湍流模糊退化
imgout1,Hatm1 = AtmoTurbulenceSim(img, 0.001)
#令k=0.0025,模拟大气湍流模糊退化
imgout2,Hatm2 = AtmoTurbulenceSim(img, 0.0025)

#显示退化结果
plt.figure(figsize=(15,8))
plt.gray()

#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Original image') 
plt.axis('off')
#模糊退化结果
plt.subplot(1,3,2); plt.imshow(imgout1,vmin=0,vmax=255)
plt.title('atmospheric turbulence, k=0.001') 
plt.axis('off')
#模糊退化结果
plt.subplot(1,3,3); plt.imshow(imgout2,vmin=0,vmax=255,cmap='gray')
plt.title('atmospheric turbulence, k=0.0025') 
plt.axis('off')

plt.tight_layout()
plt.show()
#----------------------------------------------------

In [4]:
#io.imsave('aerial_image0025.png',imgout2)

## 2.运动模糊图像退化模型
- 匀速直线运动引起的图像模糊
- #### 先运行函数定义cell装载函数，才能调用

In [4]:
def MotionBlurSim(image, Te, xa, yb):
    """
    模拟x,y方向匀速直线运动模糊退化
    输入参数:
        image - 灰度图像或RGB彩色图像
           Te - 成像设备曝光时间
        xa,yb - 在曝光时间T内像点在水平和垂直方向上的移动量
    输出参数:
       imgout - 退化图像
          Hmb - 匀速直线运动模糊退化函数
    """
    #获取图像高/宽
    rows,cols = image.shape[0:2]
    
    #采用'reflect'方式扩展图像(下面扩展rows行,右面扩张cols列)
    if image.ndim==3:
        #彩色图像
        imgex = np.pad(image,((0,rows),(0,cols),(0,0)),mode='reflect')
    elif image.ndim==2:
        #灰度图像        
        imgex = np.pad(image,((0,rows),(0,cols)),mode='reflect')
    
    #计算扩展图像的DFT,并中心化
    img_dft = fftshift(fft2(imgex,axes=(0,1)),axes=(0,1))
    
    #生成运动模糊退化函数
    #构建频域平面坐标网格数组，坐标轴定义v列向/u行向
    v = np.arange(-cols, cols) 
    u = np.arange(-rows, rows)
    Va, Ua = np.meshgrid(v, u)

    temp = np.pi * (Ua * yb + Va * xa)
    #当temp=0,sin(temp)/temp = 1
    Hmb = Te * np.ones((2*rows,2*cols)).astype(complex) #用complex 替换已废弃的np.complex
    indx = np.nonzero(temp) 
    Hmb[indx] = np.exp(-1j * temp[indx])* Te * np.sin(temp[indx])/temp[indx]
 
    if image.ndim==3:
        #彩色图像把H串接成三维数组
        Hmb = np.dstack((Hmb, Hmb, Hmb)) 
        
    #计算图像DFT与运动模糊退化函数的点积
    Gp= img_dft * Hmb
    #去中心化
    Gp = ifftshift(Gp,axes=(0,1))
    
    #DFT反变换并取实部
    imgp = np.real(ifft2(Gp,axes=(0,1)))
    #把输出图像的数据格式转换为uint8
    imgp = np.uint8(np.clip(imgp,0,255))
    #截取imgp左上角与原图像大小相等的区域作为输出
    imgout = imgp[0:rows,0:cols]
    
    return imgout,Hmb
#---------------------------------------------------

In [ ]:
#采用自编函数MotionBlurSim模拟匀速直线运动图像模糊退化

#读入一幅图片
img = io.imread('./imagedata/cameraman.tif')

#运动模糊退化参数
Te = 1  #曝光时间
xa = 0.02; yb = 0.02  #运动速度

imgout,Hmb = MotionBlurSim(img,Te,xa,yb)
#imgout,Hmb2 = MotionBlurSim(imgout,Te,0,-0.03)
#Hmb = Hmb*Hmb2 

#显示退化结果
plt.figure(figsize=(16,8))

#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255,cmap='gray')
plt.title('Original image') 
plt.axis('off')
#运动模糊退化结果
plt.subplot(1,3,2); plt.imshow(imgout,vmin=0,vmax=255,cmap='gray')
plt.title('motion blurred image') 
plt.axis('off')
#运动模糊退化函数
plt.subplot(1,3,3); plt.imshow(np.abs(Hmb),cmap='gray')
plt.title(' the functuion of MotionBlur') 
plt.axis('off')

plt.show()
#----------------------------------------------------

## 7.2 逆滤波图像复原
### 7.2.1 直接逆滤波
### 7.2.2 加窗逆滤波

## 示例：逆滤波图像复原
- #### 先运行函数定义cell装载函数，才能调用

In [10]:
def WinInvFilter(image, H, radius):
    """
    加窗逆滤波复原图像,Windowed inverse filtering
    输入参数:
        image - 灰度图像或RGB彩色图像
            H - 图像退化函数的DFT,行/列数是image的2倍
       radius - 加窗半径(截止频率)
    输出参数:
        imgout- 复原的图像
    """
    #获取图像高/宽
    rows,cols = image.shape[0:2]
    
    #采用'reflect'方式扩展图像(下面扩展rows行,右面扩张cols列)
    if image.ndim==3:
        #彩色图像
        imgex = np.pad(image,((0,rows),(0,cols),(0,0)),mode='reflect')
    elif image.ndim==2:
        #灰度图像        
        imgex = np.pad(image,((0,rows),(0,cols)),mode='reflect')
        
    #计算扩展图像的DFT,并中心化
    img_dft = fftshift(fft2(imgex,axes=(0,1)),axes=(0,1))
    
    #加窗逆滤波Windowed inverse filtering
    Gp = img_dft.copy() 

    #构建频域平面坐标网格数组，坐标轴定义v列向/u行向
    v = np.arange(-cols, cols) 
    u = np.arange(-rows, rows)
    Va, Ua = np.meshgrid(v, u)
    D2 = Ua ** 2 + Va ** 2
    #计算圆形窗区域内的元素索引
    indx = D2 <= radius ** 2
    Gp[indx] = img_dft[indx]/(Hatm[indx] + np.finfo(np.float32).eps)    
    #去中心化
    Gp = ifftshift(Gp,axes=(0,1))
    #DFT反变换,并取实部
    imgp = np.real(ifft2(Gp,axes=(0,1)))
    #把输出图像的数据格式转换为uint8
    imgp = np.uint8(np.clip(imgp,0,255))
    #截取imgp左上角与原图像大小相等的区域作为输出
    imgout = imgp[0:rows,0:cols]
    
    return imgout
#----------------------------------

In [ ]:
#模拟大气湍流图像退化及逆滤波复原

#读入一幅航拍图片
img = io.imread('./imagedata/aerial_image.png')

#令k=0.001模拟大气湍流模糊退化
img_deg, Hatm = AtmoTurbulenceSim(img,0.001)

#向退化图像中添加向图像中添加均值为0、方差为0.001的高斯噪声
img_deg_noi = util.random_noise(img_deg,mode='gaussian',var=0.001)
img_deg_noi = util.img_as_ubyte(img_deg_noi)

#对不加噪退化图像进行直接逆滤波
#令窗口半径极大,相当于不加窗直接逆滤波
img_res1 = WinInvFilter(img_deg, Hatm, np.finfo(np.float32).max)

#对不加噪退化图像进行加窗逆滤波Windowed inverse filtering
#截止频率分别radius为120,180,220
img_res2 = WinInvFilter(img_deg, Hatm, 120)
img_res3 = WinInvFilter(img_deg, Hatm, 180)
img_res4 = WinInvFilter(img_deg, Hatm, 220)

#对加噪退化图像进行加窗逆滤波Windowed inverse filtering
img_res5 = WinInvFilter(img_deg_noi,Hatm, 120)
img_res6 = WinInvFilter(img_deg_noi, Hatm, 180)

#显示结果
plt.figure(figsize=(16,9))
plt.gray()

#原图像
plt.subplot(3,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Original image') 
plt.axis('off')
#退化图像
plt.subplot(3,3,2); plt.imshow(img_deg,vmin=0,vmax=255)
plt.title('atmospheric turbulence, k=0.001') 
plt.axis('off')
#退化图像加噪
plt.subplot(3,3,3); plt.imshow(img_deg_noi,vmin=0,vmax=255)
plt.title('add Gaussian noise') 
plt.axis('off')
#直接逆滤波结果
plt.subplot(3,3,4); plt.imshow(img_res1,vmin=0,vmax=255)
plt.title('Direct inverse filtering')
plt.axis('off')
#加窗逆滤波结果
plt.subplot(3,3,5); plt.imshow(img_res2,vmin=0,vmax=255)
plt.title('Windowed inverse filtering,radius=120')
plt.axis('off')
plt.subplot(3,3,6); plt.imshow(img_res3,vmin=0,vmax=255)
plt.title('Windowed inverse filtering,radius=180')
plt.axis('off')
plt.subplot(3,3,7), plt.imshow(img_res4,vmin=0,vmax=255)
plt.title('Windowed inverse filtering,radius=220')
plt.axis('off')
#含噪图像加窗逆滤波结果
plt.subplot(3,3,8); plt.imshow(img_res5,vmin=0,vmax=255)
plt.title('noise,Windowed inverse filtering,radius=120')
plt.axis('off')
plt.subplot(3,3,9); plt.imshow(img_res6,vmin=0,vmax=255)
plt.title('noise,Windowed inverse filtering,radius=180')
plt.axis('off')

plt.tight_layout()
plt.show()
#----------------------------------------------

## 7.3 维纳滤波图像复原

In [ ]:
#采用维纳滤波复原图像Deblur image using Wiener filter

#读入一幅航拍图片
img = io.imread('./imagedata/aerial_image.png')

#令k=0.0025模拟大气湍流模糊退化
img_deg, Hatm = AtmoTurbulenceSim(img,0.0025)

#向退化图像中添加向图像中添加均值为0、方差为0.001的高斯噪声
img_deg_noi = util.random_noise(img_deg,mode='gaussian',var=0.001)
img_deg_noi = util.img_as_ubyte(img_deg_noi)

#获取退化图像的高/宽
rows,cols = img.shape[0:2]
    
#采用'reflect'方式扩展图像(下面扩展rows行,右面扩张cols列)
if img.ndim==3:
    #彩色图像
    imgex = np.pad(img_deg_noi,((0,rows),(0,cols),(0,0)),mode='reflect')
elif img.ndim==2:
    #灰度图像        
    imgex = np.pad(img_deg_noi,((0,rows),(0,cols)),mode='reflect')
        
#计算扩展图像的DFT并中心化
img_dft = fftshift(fft2(imgex,axes=(0,1)),axes=(0,1))    

#计算维纳滤波复原图像的频谱
NSR = 0.005
Gp = img_dft * np.conj(Hatm)/(np.abs(Hatm)**2 + NSR + np.finfo(np.float32).eps)
#去中心化
Gp = ifftshift(Gp,axes=(0,1))
#DFT反变换并取实部
imgp = np.real(ifft2(Gp,axes=(0,1)))
#把输出图像的数据格式转换为uint8
imgp = np.uint8(np.clip(imgp,0,255))
#截取imgp左上角与原图像大小相等的区域作为输出
img_res = imgp[0:rows,0:cols]

#显示结果
plt.figure(figsize=(15,15))
#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255,cmap='gray')
plt.title('Original image') 
plt.axis('off')
#退化图像加噪
plt.subplot(1,3,2); plt.imshow(img_deg_noi,vmin=0,vmax=255,cmap='gray')
plt.title('add Gaussian noise') 
plt.axis('off')
#含噪图像加Wiener滤波结果
plt.subplot(1,3,3); plt.imshow(img_res,vmin=0,vmax=255,cmap='gray')
plt.title('deblur image using Wiener filter')
plt.axis('off')

plt.tight_layout()
plt.show()
#---------------------------------------------------------------

## 7.4 约束最小二乘滤波图像复原

## 示例：约束最小二乘滤波复原图像

In [ ]:
#约束最小二乘滤波图像复原Constrained least squares filtering
#读入一幅图片
img = io.imread('./imagedata/book_lunyu.jpg')
#获取图像高/宽
rows,cols = img.shape[0:2]

#运动模糊退化参数
Te = 1  #曝光时间
xa = 0.02; yb = 0.02  #运动速度

img_deg,Hmb = MotionBlurSim(img,Te,xa,yb)

#向退化图像中添加向图像中添加均值为0、方差为0.001的高斯噪声
img_deg = util.random_noise(img_deg,mode='gaussian',var=0.001)
img_deg = util.img_as_ubyte(img_deg)

#Laplace算子
lp = np.array([[0,1,0],[1,-4,1],[0,1,0]])    
lpex = np.pad(lp,((0,2*rows-3),(0,2*cols-3)),mode='constant')
#计算扩展Laplace算子的DFT,并中心化
lpex_dft = fftshift(fft2(lpex,axes=(0,1)),axes=(0,1)) 
if img_deg.ndim==3:
    #彩色图像把H串接成三维数组
    lpex_dft = np.dstack((lpex_dft, lpex_dft, lpex_dft))

#采用'reflect'方式扩展图像(下面扩展rows行,右面扩张cols列)
if img_deg.ndim==3:
    #彩色图像
    imgex = np.pad(img_deg,((0,rows),(0,cols),(0,0)),mode='reflect')
elif img_deg.ndim==2:
    #灰度图像        
    imgex = np.pad(img_deg,((0,rows),(0,cols)),mode='reflect')
    
#计算扩展图像的DFT,并中心化
img_dft = fftshift(fft2(imgex,axes=(0,1)),axes=(0,1))    

#计算约束最小二乘滤波复原图像的频谱
gama = 0.01
Gp = img_dft * np.conj(Hmb)/(np.abs(Hmb)**2 + gama * np.abs(lpex_dft)**2 \
                                            + np.finfo(np.float32).eps)
#去中心化
Gp = ifftshift(Gp,axes=(0,1))
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp,axes=(0,1)))
#把输出图像的数据格式转换为uint8
imgp = np.uint8(np.clip(imgp,0,255))
#截取imgp左上角与原图像大小相等的区域作为输出
img_res = imgp[0:rows,0:cols]

#计算维纳滤波复原图像的频谱
NSR = 0.01
Gpw = img_dft * np.conj(Hmb)/(np.abs(Hmb)**2 + NSR + np.finfo(np.float32).eps)
#去中心化
Gpw = ifftshift(Gpw,axes=(0,1))
#DFT反变换,并取实部
imgpw = np.real(ifft2(Gpw,axes=(0,1)))
#把输出图像的数据格式转换为uint8
imgpw = np.uint8(np.clip(imgpw,0,255))
#截取imgpw左上角与原图像大小相等的区域作为输出
img_resw = imgpw[0:rows,0:cols]

#显示结果
plt.figure(figsize=(18,9))

#原图像
plt.subplot(1,4,1), plt.imshow(img,vmin=0,vmax=255)
plt.title('Original image') 
plt.axis('off')
#退化图像
plt.subplot(1,4,2), plt.imshow(img_deg,vmin=0,vmax=255)
plt.title('Blurred image ') 
plt.axis('off')
#约束最小二乘滤波结果
plt.subplot(1,4,3), plt.imshow(img_res,vmin=0,vmax=255)
plt.title('Deblur image using Constrained LS filtering')
plt.axis('off')

#Wiener滤波结果
plt.subplot(1,4,4), plt.imshow(img_resw,vmin=0,vmax=255)
plt.title('Deblur image using Wiener filter')
plt.axis('off')

plt.show()

## Scikit-image图像复原

In [ ]:
from skimage import color, data, restoration
from scipy.signal import convolve2d

img1 = color.rgb2gray(data.astronaut())


psf = np.ones((5, 5)) / 25
img = convolve2d(img1, psf, 'same')
img += 0.1 * img.std() * np.random.standard_normal(img.shape)
deconvolved_img,a = restoration.unsupervised_wiener(img, psf)

# Restore Image using Richardson-Lucy algorithm
deconvolved_RL = restoration.richardson_lucy(img, psf, num_iter = 2) # iterations 已废弃

#显示结果
plt.figure(figsize=(15,15))
#原图像
plt.subplot(1,4,1), plt.imshow(img1,cmap='gray')
plt.title('Original image') 
plt.axis('off')
#退化图像加噪
plt.subplot(1,4,2), plt.imshow(img,cmap='gray')
plt.title('add Gaussian noise') 
plt.axis('off')
#含噪图像加Wiener滤波结果,K=0.02
plt.subplot(1,4,3), plt.imshow(deconvolved_img,cmap='gray')
plt.title('deblur image using Wiener filter')
plt.axis('off')

#含噪图像加Wiener滤波结果,K=0.02
plt.subplot(1,4,4), plt.imshow(deconvolved_RL,cmap='gray')
plt.title('deblur image using ichardson-Lucy algorithm')
plt.axis('off')

plt.show()

## 7.5 图像修复

- Restores the selected region in an image using the region neighborhood.
- dst=cv.inpaint(src,inpaintMask,inpaintRadius,flags[, dst])

- Parameters
  - src，Input 8-bit, 16-bit unsigned or 32-bit float 1-channel or 8-bit 3-channel image.
  - inpaintMask，Inpainting mask, 8-bit 1-channel image. Non-zero pixels indicate the area that needs to be inpainted.
  - dst，Output image with the same size and type as src .
  - inpaintRadius，Radius of a circular neighborhood of each point inpainted that is considered by the algorithm.
  - flags，Inpainting method that could be cv.INPAINT_NS or cv.INPAINT_TELEA
- The function reconstructs the selected image area from the pixel near the area boundary. The function may be used to remove dust and scratches from a scanned photo, or to remove undesirable objects from still images or video. See http://en.wikipedia.org/wiki/Inpainting for more details.
  - src，输入的单通道或三通道图像；
  - inpaintMask，图像的掩码，单通道图像，大小跟原图像一致，inpaintMask图像上除了需要修复的部分之外其他部分的像素值全部为0；
  - dst，输出的经过修复的图像；
  - inpaintRadius，修复算法取的邻域半径，用于计算当前像素点的差值；
  - flags，修复算法，有两种：INPAINT_NS 和I NPAINT_TELEA；
  
  
- skimage.restoration.inpaint_biharmonic(image, mask, multichannel=False)
- Parameters
  - image,(M[, N[, …, P]][, C]) ndarray Input image.
  - mask,(M[, N[, …, P]]) ,ndarray, Array of pixels to be inpainted. Have to be the same shape as one of the ‘image’ channels. Unknown pixels have to be represented with 1, known pixels - with 0.
  - multichanne,lboolean, optional,If True, the last image dimension is considered as a color channel, otherwise as spatial.

- Image inpainting 图像修补最新综述
- https://mp.weixin.qq.com/s?__biz=MzIwMTE1NjQxMQ==&mid=2247488547&idx=3&sn=6e6dc070c1593b0dc87acaaa2a447daf

## 示例：修复图像划痕

In [ ]:
#OpenCV: 图像修复

#读入瑕疵图像
img = cv.imread('.\imagedata\lake_crack.png',cv.IMREAD_GRAYSCALE)
#读入瑕疵区域掩膜图像
mask = cv.imread('.\imagedata\lake_crack_mask.png',cv.IMREAD_GRAYSCALE)

#采用INPAINT_NS方式
img_ns = cv.inpaint(img,mask,5,cv.INPAINT_NS)
#采用INPAINT_TELEA
img_telea = cv.inpaint(img,mask,5,cv.INPAINT_TELEA)

#采用Scikit-image函数
image_sk = restoration.inpaint_biharmonic(img, mask)
image_sk = util.img_as_ubyte(image_sk)

#显示结果
plt.figure(figsize=(18,8))
plt.gray()

#瑕疵图像
plt.subplot(1,5,1), plt.imshow(img,vmin=0,vmax=255)
plt.title('Original image') 
plt.axis('off')
#瑕疵区域掩膜图像
plt.subplot(1,5,2), plt.imshow(mask,vmin=0,vmax=255)
plt.title('mask,the area that needs to be inpainted')
plt.axis('off')
#INPAINT_NS方式修复结果
plt.subplot(1,5,3), plt.imshow(img_ns,vmin=0,vmax=255)
plt.title('INPAINT_NS')
plt.axis('off')
#INPAINT_TELEA方式修复结果
plt.subplot(1,5,4), plt.imshow(img_telea,vmin=0,vmax=255)
plt.title('INPAINT_TELEA')
plt.axis('off') 
#Scikit-image函数修复结果
plt.subplot(1,5,5), plt.imshow(image_sk,vmin=0,vmax=255)
plt.title('Scikit-image')
plt.axis('off')

plt.show()

### 示例: 采用图像修复方法图像中指定区域物体 -Remove Objects in Image Using Inpainting

In [ ]:
#OpenCV: 调用图像修复函数去除图像中指定区域物体
#读入图像
img = cv.imread('.\imagedata\liftingbody.png',cv.IMREAD_GRAYSCALE)
#读入要去除区域的掩膜图像
mask = cv.imread('.\imagedata\liftingbody_mask.png', cv.IMREAD_GRAYSCALE)

#采用INPAINT_NS方式
img_rem = cv.inpaint(img,mask,5,cv.INPAINT_NS)

#显示结果
plt.figure(figsize=(16,8))
#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255,cmap='gray')
plt.title('Original image') 
plt.axis('off')
#移除指定兴趣区域目标后的图像
plt.subplot(1,3,2); plt.imshow(img_rem,vmin=0,vmax=255,cmap='gray')
plt.title('Removes Objects in Image')
plt.axis('off')
#移除目标区域的掩膜图像
plt.subplot(1,3,3); plt.imshow(mask,vmin=0,vmax=255,cmap='gray')
plt.title('mask,the area that needs to be Removed')
plt.axis('off')

plt.show()

# The end 